In [12]:
using HDF5
using Plots
#using GR
using Statistics
using DelimitedFiles

function read_snap(filename :: String)
    
    pos  = h5read(filename,"PartType0/Coordinates")
#    Bfl  = h5read(filename,"PartType0/Bfield") 
    Vel  = h5read(filename,"PartType0/Velocities")
#    alp  = h5read(filename,"PartType0/EPalpha")
#    bet  = h5read(filename,"PartType0/EPbeta")
#    divB = h5read(filename,"PartType0/divB")
    #Ids  = h5read(filename,"PartType0/ParticleIDs")
    h    = h5read(filename,"PartType0/SmoothingLengths")
    rho  = h5read(filename,"PartType0/Densities")
    u    = h5read(filename,"PartType0/InternalEnergies")
    P    = h5read(filename,"PartType0/Pressures")
#    Ga   = h5read(filename,"PartType0/divB")
    head = h5readattr(filename,"Header")
    
    print("Leyendo ",filename," at time: ",head["Time"],"\n")
    x = pos[1,:]
    y = pos[2,:]
    z = pos[3,:]
#    bx = Bfl[1,:]
#    by = Bfl[2,:]
#    bz = Bfl[3,:]
    Vx = Vel[1,:]
    Vy = Vel[2,:]
    Vz = Vel[3,:]
    
    v2 = Vx.*Vx.+Vy.*Vy.+Vz.*Vz
#    b2 = by.*by.+by.*by.+bz.*bz
    
    Npart=size(b2,1)
    
    NedivB=h .*  divB ./sqrt.(b2)
    
    print("Edivb mean: ",mean(abs.(NedivB)),"\n")
    print("Min x:",minimum(x)," / Max x:",maximum(x),"\n")
    print("Min y:",minimum(y)," / Max y:",maximum(y),"\n")
    print("Min z:",minimum(z)," / Max z:",maximum(z),"\n")
    print("Min h:",minimum(h)," / Max h:",maximum(h),"\n")
#    print("Min B2:",minimum(b2)," / Max B2:",maximum(b2),"\n")
    (Dict(:H => head, :x=>pos, :bfl=> Bfl, :b2=>b2, :v=>Vel, :v2=>v2, 
            :divB=>divB, :rho=>rho, :hsml=>h, #:bet=> bet, :alp=>alp, 
            :Pres=> P, u=>u, :Npart=> Npart, :Gau=>Ga))
end

function do_heat(data,what,Nmax)
    #Nmax=128
    #x=x.-minimum(x)
    #y=y.-minimum(y)
    #b2=data[what]
    b2=what
    Npart=size(b2,1)
    Lbox=1.0#+maximum(x)
    A = ones((Nmax, Nmax))*minimum(b2)
    grid = ones((Nmax, Nmax))
    #grid = [Set{Int}() for x=1:Nmax, y=1:Nmax];
    #insert(grid) = p -> push!(grid[trunc(Int, p.x/Lbox*Nmax), trunc(Int, p.y/Lbox*Nmax)], p.n);
    #foreach(insert(grid), data)
    x=data[:x][2,:]
    y=data[:x][1,:]
    for ind = 1:Npart
        i,j = trunc(Int, x[ind]/Lbox*Nmax+1), trunc(Int, y[ind]/Lbox*Nmax+1)
        A[i,j] += b2[ind]
        grid[i,j] += 1 
    end
    #AA=log10.(A./grid)
    AA=(A./grid)
    (AA)
end

function four_plt(file)
#file="OrszagTangVortex_0003.hdf5"
  if isfile(file) 
   run=read_snap(file)
   #run=read_snap("OT.NR.VP.00.hdf5")
   Nmax =200
   EdivB=run[:hsml].*run[:divB] ./ sqrt.(run[:b2])
   EdivB = log10.(abs.(EdivB))
   #EdivB = abs.(run[:Gau])
   #EdivB = run[:Pres]./(sqrt.(run[:b2]/(8*3.14)))
   bb=run[:bfl]
   gr()
   AA   =do_heat(run,(run[:rho]),Nmax)
   h1=heatmap(1:Nmax,1:Nmax,AA,color=cgrad([:blue,:green,:yellow,:red]),clim=(0.1,0.5),colorbar=:left)
   AA   =do_heat(run,(run[:Pres]),Nmax)
   h2=heatmap(1:Nmax,1:Nmax,AA,color=cgrad([:blue,:yellow,:purple]),clim=(0.01,0.7)) 
   AA   =do_heat(run,(run[:b2])./2,Nmax)
   h3=heatmap(1:Nmax,1:Nmax,AA,color=cgrad([:blue, :yellow ,:red]),clim=(0.01,0.25))
   AA   =do_heat(run,(EdivB),Nmax)
   #h4=heatmap(1:Nmax,1:Nmax,AA,color=cgrad([:blue,:white]),clim=(0.0,10.0))#,clim=(-4.0,-1.0))

   h4=heatmap(1:Nmax,1:Nmax,AA,color=cgrad([:blue,:white]),clim=(-10.0,-1.0))

   plot(h1,h2,h3,h4,layout=4,size=(1024,1024),aspect_ratio=:equal,colorbar_title=["\$Density\$" "\$Pressure\$" "\$B^2/2\$" "\$log10(err_{divB})\$"],colorbar=[:left :top :left :right])

    #savefig("Vort001.png")
  else
     println("File ",file," *** NOT FOUND ***")
    
     #savefig("Vort001.png")
   end
end

function mirror_plt(file)
    if isfile(file)
        run=read_snap(file)
        #run=read_snap("OT.NR.VP.00.hdf5")
        Nmax = 256
        Nmax2 = Nmax
        EdivB=run[:hsml].*run[:divB] ./ sqrt.(run[:b2])
        EdivB = log10.(abs.(EdivB))
        bb=run[:bfl]
        gr()
        AA   =do_heat(run,(run[:rho]),Nmax)
        #AA = AA'
        r0 = range(start=1,stop=Nmax,step=1)
        r1 = range(start=1,stop=Nmax,step=1)
        r2 = range(start=Nmax,stop=1,step=-1)
        r3 = range(start=Nmax,stop=1,step=-1)
        BB1  = AA[r0 ,r1] 
        BB2  = AA[r2, r3]
        BB = (BB1.-BB2)./(BB1.+BB2)
        BB = abs.(BB)
        println("Maximum ",maximum(BB))
        h1=heatmap(1:Nmax,1:Nmax2,BB,color=cgrad([:white,:blue,:red]),clim=(0.00001,0.05),aspect_ratio=:equal)
    else
        println("File ",file," *** NOT FOUND ***")
    end
end


function pltStat(file)
    if isfile(file)
        re=readdlm("stat_BASE.txt", skipstart=108)
        ri=readdlm(file, skipstart=108)
        eTim=re[:,2]
        eb2=re[:,35]
        edivB=re[:,36]
        eHmag=re[:,38]
        
        iTim=ri[:,2]
        ib2=ri[:,35]
        idivB=ri[:,36]
        iHmag=ri[:,38]
        
        lab=file[begin:end-15]
        
        h1=plot([eTim,iTim],[eb2,ib2],ylabel="b2",label=["Norm" lab])
        h2=plot([eTim,iTim],[edivB,idivB],ylabel="DivB",label=["Norm" lab],yaxis=:log)
        h3=plot([eTim,iTim],[eHmag,iHmag],ylabel="Hmag",label=["Norm" lab],ylim=(-0.5,0.5))
        h4=plot([eTim,iTim],[abs.(eHmag),abs.(iHmag)],ylab="|Hmag|",label=["Norm" lab])
        plot(h1,h2,h3,h4,layout=4, xlabel="Time" )
    else
        println("File ",file," *** NOT FOUND ***")
    end
end

function do_box(data,what,Nmax)
    #Nmax=128
    #x=x.-minimum(x)
    #y=y.-minimum(y)
    #b2=data[what]
    b2=what
    Npart=size(b2,1)
    Lbox=1.0#+maximum(x)
    A = ones((Nmax, Nmax, Nmax))*minimum(b2)
    grid = ones((Nmax, Nmax, Nmax))
    x=data[:x][1,:]
    y=data[:x][2,:]
    z=data[:x][3,:]
    
    for ind = 1:Npart
        i,j,k = trunc(Int, x[ind]/Lbox*Nmax+1), trunc(Int, y[ind]/Lbox*Nmax+1), trunc(Int, z[ind]/Lbox*Nmax+1)
        A[i,j,k] += b2[ind]
        grid[i,j,k] += 1 
    end
    #AA=log10.(A./grid)
    AA=(A./grid)
    (AA)
end


do_box (generic function with 1 method)

In [13]:
a=read_snap("turb-box_0000.hdf5")

Leyendo turb-box_0000.hdf5 at time: [0.0]


LoadError: UndefVarError: Bfl not defined